## Studying AuxTel banding vs Rotator Angle

Craig Lage - 11-Nov-22

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
import lsst.afw.cameraGeom.utils as camGeomUtils
from lsst.daf.butler import Butler
import lsst.geom as geom

In [ ]:
butler = Butler('/repo/oga', collections=["LATISS/raw/all", "LATISS/calib"])
camera = butler.get('camera', instrument='LATISS')

In [ ]:
def matlab_smooth(data, window):
    if window%2 == 0:
        window += 1
        print(f"Even window size adjusted to {window}")
    output = np.convolve(data, np.ones(window, dtype=float), 'valid')/window    
    r = np.arange(1, window-1, 2)
    start = np.cumsum(data[:window-1])[::2]/r
    stop = (np.cumsum(data[:-window:-1])[::2]/r)[::-1]
    return np.concatenate((start, output, stop))

def calcBanding(exp, useHalfChip=True):
    bandings = []
    amps = exp.getDetector().getAmplifiers()
    if useHalfChip:
        amps = amps[0:8]
    for amp in amps:
        overscanData = raw.image[amp.getRawSerialOverscanBBox()].array
        collapsed = np.mean(overscanData, axis=1)
        collapsed = matlab_smooth(collapsed, smoothing)
        low = np.percentile(collapsed, lowPercentile)
        high = np.percentile(collapsed, highPercentile)
        bandings.append(high - low)
    banding = np.median(bandings)
    return banding

def calcNoises(exp):
    noises = []
    for amp in exp.getDetector():
        noiseBbox = amp.getRawSerialOverscanBBox()
        noiseBbox.grow(-1*noiseBorder)
        noises.append(np.std(exp.image[noiseBbox].array))
    return noises


In [ ]:
lowPercentile = 10
highPercentile = 90
smoothing = 3
noiseBorder = 10  # number of pixels in all directions to shrink overscan for noise calc
rots = []
bandings = []
seqNums = []
rot0 = -160.0
for i, seqNum in enumerate(range(11, 171)):
    rot = rot0 + i * 2.0
    rots.append(rot)
    seqNums.append(seqNum)
    dataId = {'day_obs': 20221115, 'seq_num': seqNum, 'detector': 0}
    raw = butler.get('raw', dataId)
    banding = calcBanding(raw)
    bandings.append(banding)
    noises = calcNoises(raw)
    noise = np.mean(noises)
    print(f"Image {seqNum}: banding = {banding:.3f}, noise = {noise:.3f}")

In [ ]:
plt.title("AuxTel Banding vs Rotator Angle - 20221115")
plt.plot(rots, bandings)
plt.ylabel("Banding metric (ADU)")
plt.xlabel("Nasmyth2 rotator angle (degrees)")
plt.arrow(0, 5.0, 50,0, width=0.1,head_length = 5.0, color='green')
plt.savefig("/home/c/cslage/u/AuxTel/isr/Banding_Test_Plus_15Nov22.pdf")

In [ ]:
lowPercentile = 10
highPercentile = 90
smoothing = 3
noiseBorder = 10  # number of pixels in all directions to shrink overscan for noise calc
rots = []
bandings = []
seqNums = []
rot0 = 160.0
for i, seqNum in enumerate(range(171, 331)):
    rot = rot0 - i * 2.0
    rots.append(rot)
    seqNums.append(seqNum)
    dataId = {'day_obs': 20221115, 'seq_num': seqNum, 'detector': 0}
    raw = butler.get('raw', dataId)
    banding = calcBanding(raw)
    bandings.append(banding)
    noises = calcNoises(raw)
    noise = np.mean(noises)
    print(f"Image {seqNum}: banding = {banding:.3f}, noise = {noise:.3f}")

In [ ]:
plt.title("AuxTel Banding vs Rotator Angle - 20221115")
plt.plot(rots, bandings)
plt.ylabel("Banding metric (ADU)")
plt.xlabel("Nasmyth2 rotator angle (degrees)")
plt.arrow(0, 5.0, -50,0, width=0.1,head_length = 5.0, color='green')
plt.savefig("/home/c/cslage/u/AuxTel/isr/Banding_Test_Minus_15Nov22.pdf")

In [ ]:
lowPercentile = 10
highPercentile = 90
smoothing = 3
noiseBorder = 10  # number of pixels in all directions to shrink overscan for noise calc
rots = []
bandings = []
seqNums = []
rot0 = -160.0
for i, seqNum in enumerate(range(331, 491)):
    rot = rot0 + i * 2.0
    rots.append(rot)
    seqNums.append(seqNum)
    dataId = {'day_obs': 20221115, 'seq_num': seqNum, 'detector': 0}
    raw = butler.get('raw', dataId)
    banding = calcBanding(raw)
    bandings.append(banding)
    noises = calcNoises(raw)
    noise = np.mean(noises)
    print(f"Image {seqNum}: banding = {banding:.3f}, noise = {noise:.3f}")

In [ ]:
plt.title("AuxTel Banding vs Rotator Angle - 20221115")
plt.plot(rots, bandings)
plt.ylabel("Banding metric (ADU)")
plt.xlabel("Nasmyth2 rotator angle (degrees)")
plt.arrow(0, 5.0, 50,0, width=0.1,head_length = 5.0, color='green')
plt.text(0, 4.0, "5 second delay between images", color='green')
plt.savefig("/home/c/cslage/u/AuxTel/isr/Banding_Test_Plus_Slow_15Nov22.pdf")

In [ ]:
for i in range(160):
    print(seqNums[i], rots[i], bandings[i])